In [ ]:
!pip3 install tensorflow-gpu==2.2.0 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/CNN_VGG/
!ls

In [ ]:
# define some variables for later usage
epochs = 50
img_size = 150
batch_size = 32

In [ ]:
data_dir = "/content/drive/MyDrive/CNN_VGG/"
TEST = '/CNN_VGG/Test/'
TRAIN = '/CNN_VGG/Train/'

In [ ]:
# import required libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
#import keras.layers.experimental.preprocessing
#from keras.models import Sequential
from keras.preprocessing import image
from mlxtend.plotting import plot_confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix

from keras.layers import Dense, Conv2D, Dropout, Flatten, Activation
from keras.layers import SeparableConv2D, MaxPool2D, BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True,vertical_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(data_dir + '/Train',target_size = (150, 150),batch_size = 8,color_mode="rgb",class_mode = 'categorical',shuffle=True)
test_set = test_datagen.flow_from_directory(data_dir + '/Test',target_size = (150, 150),batch_size = 8,color_mode="rgb",class_mode = 'categorical',shuffle=True)

In [ ]:
training_set.labels

In [ ]:
cnn = Sequential()
cnn.add(Conv2D(32, (3,3), input_shape=(150,150,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(64, (3, 3), activation = 'relu'))
cnn.add(Conv2D(64, (3, 3), activation = 'relu'))
cnn.add(Conv2D(64, (3, 3), activation = 'relu'))
cnn.add(MaxPooling2D(pool_size = (2, 2)))

cnn.add(Flatten())

cnn.add(Dense(units = 128, activation = 'relu'))
cnn.add(Dense(units = 128, activation = 'relu'))
cnn.add(Dense(units = 128, activation = 'relu'))
cnn.add(Dense(units = 128, activation = 'relu'))
cnn.add(Dense(units = 3, activation = 'softmax'))
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = cnn.fit_generator(training_set,
                         steps_per_epoch = training_set.samples//8,
                         epochs = 5,
                         validation_data = test_set,
                         validation_steps = test_set.samples//8)

In [ ]:

import cv2
import numpy as np

img = cv2.imread('/content/drive/MyDrive/CNN_VGG/Test/Fire/image_0.jpg')

plt.imshow(img)
img = cv2.resize(img,(150,150))
img = np.reshape(img,[1,150,150,3])

classes = cnn.predict_classes(img)

print(classes)



In [ ]:
cnn.summary()

In [ ]:
N = 5
plt.figure(figsize=(12,6))
plt.plot(np.arange(0, N), history.history['loss'], label="train_loss")
plt.plot(np.arange(0, N), history.history['val_loss'],label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title('Model Train vs Validation Loss')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="best")
plt.show()

In [ ]:
test_set.labels

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:
test_data = []
test_labels = []

for title in ['/Fire/', '/Neutral/','/Smoke/']:
        for img in (os.listdir(data_dir + 'Test/' + title)):
            img = plt.imread(data_dir + 'Test/' + title + img)
            img = cv2.resize(img, (img_size, img_size))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if title == '/Fire/':
                label = 0
            elif title == '/Neutral/':
                label = 1
            elif title == '/Smoke/':
                label = 2
            test_data.append(img)
            test_labels.append(label)
        
test_data = np.array(test_data)
test_labels = np.array(test_labels)

# get predictions
preds = cnn.predict(test_data)

acc = accuracy_score(test_labels, np.round(preds))*100

# get the confusion matrix
mat = confusion_matrix(test_labels, np.round(preds))

In [ ]:
print('Train Metric:')
print('Train accuracy: {}%'.format(np.round(history.history['accuracy'][-1]*100, 2)))

#model.fit(X=x_train, Y=y_train, epoch=2)
model.evaluate(training_set)
image_index=444
plt.imshow(test_set[image_index].reshape(28,28),cmap='Greys')
pred=model.predict(test_set[image_index].reshape(1,28,28,1))
print(pred.argmax())
